Треба побудувати мережу для класифікації зображень з датасету FGVC Aircraft (https://pytorch.org/vision/stable/generated/torchvision.datasets.FGVCAircraft.html#torchvision.datasets.FGVCAircraft)

Завантажимо папку fgvc-aircraft-2013b звідси:
https://www.kaggle.com/datasets/seryouxblaster764/fgvc-aircraft?select=fgvc-aircraft-2013b

В описі датасету (https://www.robots.ox.ac.uk/~vgg/data/fgvc-aircraft/) зазначено, що дані вже розбиті на однакові тренувальну, валідаційну та тестову частини.

Як tagret variable оберемо manufacturer.

In [33]:
import pandas as pd
import os
from torchvision import transforms
from torch.utils.data import Dataset, DataLoader
import torch
from PIL import Image

In [46]:
import torch.nn as nn
import torch.optim as optim
from torchvision import models
from sklearn.metrics import confusion_matrix, accuracy_score
import numpy as np

In [81]:
from sklearn.preprocessing import LabelEncoder

In [97]:
from sklearn.metrics import multilabel_confusion_matrix

In [98]:
import torch.nn.functional as F

Завантажимо txt файли

In [40]:
file_paths = {
    "test": "/Users/sofiialukashevych/Desktop/exam data aircraft/fgvc-aircraft-2013b/data/images_manufacturer_test.txt",
    "train": "/Users/sofiialukashevych/Desktop/exam data aircraft/fgvc-aircraft-2013b/data/images_manufacturer_train.txt",
    "val": "/Users/sofiialukashevych/Desktop/exam data aircraft/fgvc-aircraft-2013b/data/images_manufacturer_val.txt"
}

In [82]:
#класс для свтворення датасету
class AircraftDataset(Dataset):
    def __init__(self, txt_file, root_dir, transform=None):
        self.root_dir = root_dir
        self.transform = transform
        with open(txt_file, 'r') as file:
            self.image_labels = [self._split_line(line.strip()) for line in file.readlines()]

    def _split_line(self, line):
        parts = line.split(maxsplit=1)
        if len(parts) != 2:
            raise ValueError(f"Line '{line}' is not in the expected format.")
        return parts
    def __len__(self):
        return len(self.image_labels)

    def __getitem__(self, idx):
        img_name = os.path.join(self.root_dir, self.image_labels[idx][0] + '.jpg')
        image = Image.open(img_name)
        label = self.image_labels[idx][1]

        if self.transform:
            image = self.transform(image)

        return image, label

In [83]:
class AircraftDatasetWithEncoding(AircraftDataset):
    def __init__(self, txt_file, root_dir, transform=None):
        super().__init__(txt_file, root_dir, transform)
        # Extract all labels
        all_labels = []
        for item in self.image_labels:
            if len(item) != 2:
                raise ValueError(f"Line '{' '.join(item)}' in file {txt_file} is not in the expected format.")
            all_labels.append(item[1])
        # Initialize and fit label encoder
        self.label_encoder = LabelEncoder()
        self.label_encoder.fit(all_labels)

    def __getitem__(self, idx):
        img_name, label = self.image_labels[idx]
        image = Image.open(os.path.join(self.root_dir, img_name + '.jpg'))

        if self.transform:
            image = self.transform(image)

        # Encode label
        encoded_label = self.label_encoder.transform([label])[0]
        encoded_label_tensor = torch.tensor(encoded_label, dtype=torch.long)

        return image, encoded_label_tensor

In [42]:
#трансформація картинок
transform = transforms.Compose([
    #ресайз для входу картинок у CNN
    transforms.Resize((224, 224)),  
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

In [43]:
#шляхи до файлів
root_dir = '/Users/sofiialukashevych/Desktop/exam data aircraft/fgvc-aircraft-2013b/data/images'
train_file = "/Users/sofiialukashevych/Desktop/exam data aircraft/fgvc-aircraft-2013b/data/images_manufacturer_train.txt"
val_file = "/Users/sofiialukashevych/Desktop/exam data aircraft/fgvc-aircraft-2013b/data/images_manufacturer_val.txt"
test_file = "/Users/sofiialukashevych/Desktop/exam data aircraft/fgvc-aircraft-2013b/data/images_manufacturer_test.txt"

In [84]:
#створимо датасет
train_dataset = AircraftDatasetWithEncoding(txt_file=train_file, root_dir=root_dir, transform=transform)
val_dataset = AircraftDatasetWithEncoding(txt_file=val_file, root_dir=root_dir, transform=transform)
test_dataset = AircraftDatasetWithEncoding(txt_file=test_file, root_dir=root_dir, transform=transform)

In [102]:
#виведемо першу картинку та лейбл з тренувального датасету
first_img, first_label = test_dataset[0]
first_img, first_label

(tensor([[[-0.5082, -0.5082, -0.4911,  ...,  0.2282,  0.2282,  0.2282],
          [-0.5253, -0.5082, -0.5082,  ...,  0.2282,  0.2111,  0.2282],
          [-0.5253, -0.5082, -0.5082,  ...,  0.2282,  0.2111,  0.2111],
          ...,
          [-1.0219, -1.1418, -1.7069,  ..., -0.8335, -0.8335, -1.2617],
          [-1.2274, -1.7240, -1.2617,  ..., -0.7479, -1.3302, -2.0665],
          [-1.3473, -1.5185, -1.4672,  ..., -0.3883, -1.7069, -2.1179]],
 
         [[-0.2675, -0.2675, -0.2675,  ...,  0.4853,  0.5028,  0.5028],
          [-0.2675, -0.2675, -0.2850,  ...,  0.4853,  0.4853,  0.5028],
          [-0.2500, -0.2850, -0.2850,  ...,  0.4678,  0.4853,  0.4853],
          ...,
          [-0.8803, -1.0203, -1.6155,  ..., -0.5476, -0.5126, -1.0728],
          [-1.0553, -1.6155, -1.0903,  ..., -0.3550, -1.0203, -1.9832],
          [-1.1604, -1.3354, -1.2829,  ...,  0.0476, -1.5280, -2.0357]],
 
         [[ 0.3393,  0.3219,  0.3393,  ...,  0.9145,  0.9145,  0.9319],
          [ 0.3393,  0.3393,

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

Далі перейдемо до написання нейронної мережі.
Спочатку зробимо CNN з Dropout (regularization technique).

In [85]:
#CNN з Dropout
class SimpleCNN(nn.Module):
    def __init__(self, num_classes):
        super(SimpleCNN, self).__init__()

        #згорткові шари, витягуємо ознаки
        self.features = nn.Sequential(
            #перший згортковий шар з 64 фільтрами, розмір ядра 11x11 
            #крок 4 для зменшення розмірності
            #додавання 2 для крайніх пікселів
            nn.Conv2d(3, 64, kernel_size=11, stride=4, padding=2),
            #функція активації ReLU для нелінійності
            nn.ReLU(inplace=True),  
            #Maxpooling для зменшення просторових розмірів
            nn.MaxPool2d(kernel_size=3, stride=2),  

            #другий згортковий шар з 192 фільтрами, розмір ядра 5x5 
            nn.Conv2d(64, 192, kernel_size=5, padding=2),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=2),

            #третій згортковий шар з 384 фільтрами, розмір ядра 3x3
            nn.Conv2d(192, 384, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),

            #четвертий згортковий шар з 256 фільтрами, розмір ядра 3x3
            nn.Conv2d(384, 256, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),

            #п’ятий згортковий шар з 256 фільтрами, також розмір ядра 3x3
            nn.Conv2d(256, 256, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),

            #MaxPooling для зменшення розміру перед класифікацією
            nn.MaxPool2d(kernel_size=3, stride=2),
        )

        #AdaptiveAvgPooling
         #зміна розміру карти ознак до 6x6
        self.avgpool = nn.AdaptiveAvgPool2d((6, 6)) 

        #класифікація (повністʼю зʼєднані шари)
        self.classifier = nn.Sequential(
            #Dropout для зменшення перенавчання
            nn.Dropout(), 
            #лінійний шар
            nn.Linear(256 * 6 * 6, 4096), 
            #функція активації
            nn.ReLU(inplace=True),  
            #ще один шар Dropout для регуляризації
            nn.Dropout(),  
            #ще один лінійний шар
            nn.Linear(4096, 4096),
            #останній шар з виходами num_classes
            nn.Linear(4096, num_classes),  
        )

    def forward(self, x):
        #прохід через згорткові шари
        x = self.features(x)  
        #адаптивний пулінг
        x = self.avgpool(x)
         #вирівнювання виходу для лінійних шарів
        x = torch.flatten(x, 1) 
        #прохід через лінійні шари
        x = self.classifier(x)  
        return x


In [86]:
num_classes = 30 #стільки є виробників в файлі manufacturer.txt

In [87]:
model = SimpleCNN(num_classes=num_classes)

In [88]:
epoch_loss_values = []
step_loss_values = []
#функція втрат
loss_function = nn.CrossEntropyLoss()
#оптимізатор Адам
optimizer = torch.optim.Adam(model.parameters(), lr=0.005)

Data Loaders

In [89]:
batch_size = 64  

In [90]:
#data loader для тренувального датасету
train_data_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

In [103]:
test_data_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=True)
val_data_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=True)

In [73]:
#перевіримо перший батч (виявилось, що забула додати енкодінг лейблів, 
#після цього було створено класс AircraftDatasetWithEncoding, продовжимо)
first_batch = next(iter(train_data_loader))
print(first_batch)


[tensor([[[[-0.4739, -0.4739, -0.4568,  ..., -0.5424, -0.5424, -0.5424],
          [-0.4739, -0.4739, -0.4568,  ..., -0.5596, -0.5596, -0.5596],
          [-0.4739, -0.4739, -0.4568,  ..., -0.5424, -0.5424, -0.5424],
          ...,
          [-0.9705, -1.1760, -1.6727,  ..., -0.7479, -0.5596, -1.0048],
          [-1.2617, -1.6384, -1.2274,  ..., -1.0048, -1.1418, -2.0494],
          [-1.3815, -1.5185, -1.4500,  ..., -0.4911, -1.5185, -2.1179]],

         [[-0.1275, -0.1450, -0.1275,  ..., -0.1099, -0.1099, -0.1099],
          [-0.1450, -0.1275, -0.1275,  ..., -0.1275, -0.1275, -0.1275],
          [-0.1275, -0.1275, -0.1275,  ..., -0.1275, -0.1275, -0.1099],
          ...,
          [-0.8277, -1.0203, -1.5455,  ..., -0.5301, -0.2850, -0.8627],
          [-1.1078, -1.5280, -1.0553,  ..., -0.6702, -0.7927, -1.9482],
          [-1.1604, -1.3354, -1.2479,  ..., -0.0924, -1.3004, -2.0182]],

         [[ 0.3568,  0.3219,  0.3393,  ...,  0.3568,  0.3568,  0.3568],
          [ 0.3393,  0.3393, 

In [93]:
n_epochs = 5 #спочатку тут було 50 епох, але вантажило занадто довго навіть до 10, тому будемо дивитись на 5
n_iters = len(train_data_loader)
n_tab = str(len(str(n_epochs)))

for epoch in range(n_epochs):
    epoch_loss = 0 
    
    for current_batch in train_data_loader:
        
        optimizer.zero_grad()
        
        X_batch, Y_batch = current_batch
        X_batch = X_batch.to(device)
        Y_batch = Y_batch.to(device)
        
        Y_pred = model(X_batch)
        
        loss = loss_function(Y_pred.squeeze(), torch.flatten(Y_batch))
        step_loss_values.append(loss)

        epoch_loss += loss.item() / n_iters
        
        loss.backward()
        optimizer.step()
    
    epoch_loss_values.append(epoch_loss)
            
    print(('epoch {:' + n_tab + '}: epoch_loss: {}').format(epoch+1, epoch_loss)) 

epoch 1: epoch_loss: 2.8943128405876886
epoch 2: epoch_loss: 2.9049740197523586
epoch 3: epoch_loss: 2.903167756098621
epoch 4: epoch_loss: 2.8941094650412498
epoch 5: epoch_loss: 2.902301032588167


In [95]:
del step_loss_values
del epoch_loss_values

In [100]:
#створюємо порожній масив numpy для зберігання прогнозованих міток тренувального набору даних
Y_predicted_train = np.empty([0])

#використовуємо контекст 'torch.no_grad()' для вимкнення розрахунку градієнтів під час прогнозування
with torch.no_grad():
    #ітеруємо по кожному пакету тренувального завантажувача даних
    for current_batch in train_data_loader:
        
        #розділяємо дані та мітки поточного пакету
        X_batch, Y_batch = current_batch
        #конвертуємо дані в FloatTensor та переміщуємо на пристрій
        X_batch = X_batch.type(torch.FloatTensor).to(device)
        #переміщуємо мітки на  пристрій
        Y_batch = Y_batch.to(device)

        #виконуємо пряме поширення моделі для отримання вихідних даних на основі вхідного пакету
        Y_pred = model(X_batch)
        #застосовуємо softmax для перетворення вихідних значень моделі на ймовірності класів
        Y_pred_norm = F.softmax(Y_pred, dim=1)
        #отримуємо індекс класу з найвищою ймовірністю для кожного елементу в пакеті
        Y_pred_label = Y_pred_norm.argmax(dim=1)
        #конвертуємо прогнози в масив numpy та додаємо їх до загального масиву прогнозованих міток
        Y_pred_label = Y_pred_label.detach().cpu().numpy()
        Y_predicted_train = np.append(Y_predicted_train, Y_pred_label)
        
        #видаляємо використані змінні для звільнення пам'яті
        del X_batch
        del Y_batch
        del Y_pred_label
        #очищуємо кеш
        torch.cuda.empty_cache()

#Y_predicted_train.shape


In [104]:
#створюємо порожній масив numpy для зберігання прогнозованих міток тестового набору даних
Y_predicted_test = np.empty([0])

#використовуємо контекст 'torch.no_grad()' для вимкнення розрахунку градієнтів під час прогнозування
with torch.no_grad():
    #ітеруємо по кожному пакету тестового завантажувача даних
    for current_batch in test_data_loader:
        
        #розділяємо дані та мітки поточного пакету
        X_batch, Y_batch = current_batch
        #конвертуємо дані в FloatTensor та переміщуємо на пристрій
        X_batch = X_batch.type(torch.FloatTensor).to(device)
        #переміщуємо мітки на пристрій
        Y_batch = Y_batch.to(device)

        #виконуємо пряме поширення моделі для отримання вихідних даних на основі вхідного пакету
        Y_pred = model(X_batch)
        #застосовуємо softmax для перетворення вихідних значень моделі на ймовірності класів
        Y_pred_norm = F.softmax(Y_pred, dim=1)
        #отримуємо індекс класу з найвищою ймовірністю для кожного елементу в пакеті
        Y_pred_label = Y_pred_norm.argmax(dim=1)
        #конвертуємо прогнози в масив numpy та додаємо їх до загального масиву прогнозованих міток
        Y_pred_label = Y_pred_label.detach().cpu().numpy()
        Y_predicted_test = np.append(Y_predicted_test, Y_pred_label)
        
        #видаляємо використані змінні для звільнення пам'яті
        del X_batch
        del Y_batch
        del Y_pred_label
        #очищуємо кеш
        torch.cuda.empty_cache()
        
#Y_predicted_test.shape


In [105]:
#дістаємо лейбли
Y_train_labels = [label for _, label in train_dataset]
Y_test_labels = [label for _, label in test_dataset]

In [106]:
#переводимо лейбли в масив
Y_train_labels = np.array(Y_train_labels)
Y_test_labels = np.array(Y_test_labels)

In [107]:
Y_train_labels.shape, Y_test_labels.shape

((3334,), (3333,))

In [109]:
#з файлц manufacturer.txt
labels_map={
    0: 'ATR',
    1: 'Airbus',
    2: 'Antonov',
    3: 'Beechcraft',
    4: 'Boeing',
    5: 'Bombardier Aerospace',
    6: 'British Aerospace',
    7: 'Canadair',
    8: 'Cessna',
    9: 'Cirrus Aircraft',
    10: 'Dassault Aviation',
    11: 'Dornier',
    12: 'Douglas Aircraft Company',
    13: 'Embraer',
    14: 'Eurofighter',
    15: 'Fairchild',
    16: 'Fokker',
    17: 'Gulfstream Aerospace',
    18: 'Ilyushin',
    19: 'Lockheed Corporation',
    20: 'Lockheed Martin',
    21: 'McDonnell Douglas',
    22: 'Panavia',
    23: 'Piper',
    24: 'Robin',
    25: 'Saab',
    26: 'Supermarine',
    27: 'Tupolev',
    28: 'Yakovlev',
    29: 'de Havilland',
}

In [110]:
conf_matr_train = multilabel_confusion_matrix(
    y_true=Y_train_labels.squeeze(), 
    y_pred=Y_predicted_train.astype(int)
)

for k, label in enumerate(labels_map.values()):
    print(label)
    print(conf_matr_train[k])

ATR
[[3268    0]
 [  66    0]]
Airbus
[[2900    0]
 [ 434    0]]
Antonov
[[3300    0]
 [  34    0]]
Beechcraft
[[3267    0]
 [  67    0]]
Boeing
[[   0 2601]
 [   0  733]]
Bombardier Aerospace
[[3301    0]
 [  33    0]]
British Aerospace
[[3201    0]
 [ 133    0]]
Canadair
[[3200    0]
 [ 134    0]]
Cessna
[[3201    0]
 [ 133    0]]
Cirrus Aircraft
[[3301    0]
 [  33    0]]
Dassault Aviation
[[3267    0]
 [  67    0]]
Dornier
[[3300    0]
 [  34    0]]
Douglas Aircraft Company
[[3201    0]
 [ 133    0]]
Embraer
[[3101    0]
 [ 233    0]]
Eurofighter
[[3301    0]
 [  33    0]]
Fairchild
[[3301    0]
 [  33    0]]
Fokker
[[3234    0]
 [ 100    0]]
Gulfstream Aerospace
[[3267    0]
 [  67    0]]
Ilyushin
[[3301    0]
 [  33    0]]
Lockheed Corporation
[[3266    0]
 [  68    0]]
Lockheed Martin
[[3300    0]
 [  34    0]]
McDonnell Douglas
[[3102    0]
 [ 232    0]]
Panavia
[[3300    0]
 [  34    0]]
Piper
[[3301    0]
 [  33    0]]
Robin
[[3301    0]
 [  33    0]]
Saab
[[3267    0]
 [  67

In [111]:
conf_matr_test = multilabel_confusion_matrix(
    y_true=Y_test_labels.squeeze(), 
    y_pred=Y_predicted_test.astype(int)
)

for k, label in enumerate(labels_map.values()):
    print(label)
    print(conf_matr_test[k])

ATR
[[3266    0]
 [  67    0]]
Airbus
[[2900    0]
 [ 433    0]]
Antonov
[[3300    0]
 [  33    0]]
Beechcraft
[[3267    0]
 [  66    0]]
Boeing
[[   0 2599]
 [   0  734]]
Bombardier Aerospace
[[3300    0]
 [  33    0]]
British Aerospace
[[3200    0]
 [ 133    0]]
Canadair
[[3200    0]
 [ 133    0]]
Cessna
[[3199    0]
 [ 134    0]]
Cirrus Aircraft
[[3299    0]
 [  34    0]]
Dassault Aviation
[[3266    0]
 [  67    0]]
Dornier
[[3300    0]
 [  33    0]]
Douglas Aircraft Company
[[3200    0]
 [ 133    0]]
Embraer
[[3100    0]
 [ 233    0]]
Eurofighter
[[3299    0]
 [  34    0]]
Fairchild
[[3299    0]
 [  34    0]]
Fokker
[[3233    0]
 [ 100    0]]
Gulfstream Aerospace
[[3266    0]
 [  67    0]]
Ilyushin
[[3299    0]
 [  34    0]]
Lockheed Corporation
[[3267    0]
 [  66    0]]
Lockheed Martin
[[3300    0]
 [  33    0]]
McDonnell Douglas
[[3100    0]
 [ 233    0]]
Panavia
[[3300    0]
 [  33    0]]
Piper
[[3300    0]
 [  33    0]]
Robin
[[3299    0]
 [  34    0]]
Saab
[[3267    0]
 [  66

In [112]:
#визначаємо функцію для створення великої матриці невідповідностей
def large_confusion_matrix(y_true, y_pred, num_labels, norm=False):
    #ініціалізуємо матрицю нулів
    output_matrix = np.zeros([num_labels, num_labels]).astype(int)

    #проходимо по кожному елементу в реальних та прогнозованих мітках
    for j in range(len(y_true)):
        #збільшуємо відповідну комірку в матриці
        output_matrix[int(y_true[j]), int(y_pred[j])] += 1

    if norm:
        #конвертуємо матрицю в тип float для нормалізації
        output_matrix = output_matrix.astype(float)

        #проходимо по кожному рядку матриці
        for j in range(num_labels):
            #обчислюємо суму значень в рядку
            row_sum = np.sum(output_matrix[j,])
            #якщо сума більша за 0, то нормалізуємо рядок
            if row_sum > 0:
                output_matrix[j,] /= row_sum
        output_matrix = np.round(output_matrix, 2)

    return output_matrix


In [115]:
conf_matrix = large_confusion_matrix(Y_test_labels, Y_predicted_test, 30, norm=True)

In [116]:
cfm_test = large_confusion_matrix(
    y_true=Y_test_labels, 
    y_pred=Y_predicted_test.astype(int), 
    num_labels=30
)

In [117]:
correct_predictions = np.sum(np.diag(cfm_test))

total_test_samples = len(Y_test_labels) 
accuracy = correct_predictions / total_test_samples

print("Accuracy:", accuracy)

Accuracy: 0.22022202220222023


In [119]:
conf_matrix_train = large_confusion_matrix(Y_train_labels, Y_predicted_train, 30, norm=True)

In [120]:
#використовуємо функцію large_confusion_matrix для створення матриці помилок на основі тренувальних даних
cfm_train = large_confusion_matrix(
    y_true=Y_train_labels,  #реальні мітки з тренувального набору даних
    y_pred=Y_predicted_train.astype(int),  #прогнозовані мітки з тренувального набору даних
    num_labels=30 
)

In [121]:
#обчислюємо кількість правильних прогнозів
#np.diag витягує діагональні елементи матриці (правильні прогнози)
correct_predictions = np.sum(np.diag(cfm_train))

#ділимо кількість правильних прогнозів на загальну кількість тренувальних зразків
total_train_samples = len(Y_train_labels)  #загальна кількість тренувальних зразків
accuracy = correct_predictions / total_train_samples

# Друкуємо розраховану точність.
print("Accuracy:", accuracy)

Accuracy: 0.21985602879424115


In [122]:
#функція для обчислення точності
def calculate_accuracy(y_true, y_pred):
    #розрахунок кількості правильних прогнозів
    correct_predictions = np.sum(y_true == y_pred)
    #визначення загальної кількості зразків
    total_samples = len(y_true)
    #повернення відношення правильних прогнозів до загальної кількості зразків
    return correct_predictions / total_samples

#розрахунок точності для навчального набору даних
#припустимо, що Y_predicted_train та Y_train_labels доступні та мають відповідні форми
train_accuracy = calculate_accuracy(Y_train_labels, Y_predicted_train.astype(int))

#розрахунок точності для тестового набору даних
#припустимо, що Y_predicted_test та Y_test_labels доступні та мають відповідні форми
test_accuracy = calculate_accuracy(Y_test_labels, Y_predicted_test.astype(int))

print("Training Accuracy:", train_accuracy)
print("Test Accuracy:", test_accuracy)

#перевірка на перенавчання (overfitting)
if train_accuracy > test_accuracy:
    if train_accuracy - test_accuracy > 0.1:  
        print("Potential overfitting detected.")
    else:
        print("Slight difference in accuracies but might not be overfitting.")
else:
    print("No significant overfitting detected.")


Training Accuracy: 0.21985602879424115
Test Accuracy: 0.22022202220222023
No significant overfitting detected.


Ситуація дуже погана, точність жахлива, але радує що однаково жахлива на обох датасетах, може коли буде краще,
то стане однаково краще для обох.
Додамо в нашу модель layer normalization.

In [124]:
class AdjustedCNN(SimpleCNN):
    def __init__(self, num_classes):
        super(AdjustedCNN, self).__init__(num_classes)
        
        #згорткові шари
        self.features = nn.Sequential(
            #перший згортковий шар з 64 фільтрами, розмір ядра 11x11
            nn.Conv2d(3, 64, kernel_size=11, stride=4, padding=2),
            nn.ReLU(inplace=True),
            #нормалізація шару після ReLU
            nn.LayerNorm([64, 55, 55]),   
            nn.MaxPool2d(kernel_size=3, stride=2),
            
            #другий згортковий шар з 192 фільтрами, розмір ядра 5x5
            nn.Conv2d(64, 192, kernel_size=5, padding=2),
            nn.ReLU(inplace=True),
            #нормалізація шару
            nn.LayerNorm([192, 27, 27]),  
            nn.MaxPool2d(kernel_size=3, stride=2),
            
            #третій згортковий шар з 384 фільтрами, розмір ядра 3x3
            nn.Conv2d(192, 384, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            #нормалізація шару
            nn.LayerNorm([384, 13, 13]),  
            
            #четвертий згортковий шар з 256 фільтрами
            nn.Conv2d(384, 256, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            #нормалізація шару
            nn.LayerNorm([256, 13, 13]),  
            
            #п’ятий згортковий шар з 256 фільтрами
            nn.Conv2d(256, 256, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            #нормалізація шару
            nn.LayerNorm([256, 13, 13]),  
            nn.MaxPool2d(kernel_size=3, stride=2),
        )
        
        #пулінг
        self.avgpool = nn.AdaptiveAvgPool2d((6, 6))
        
        #класифікація
        self.classifier = nn.Sequential(
            #Dropout для зниження перенавчання
            nn.Dropout(),  
            #лінійний шар
            nn.Linear(256 * 6 * 6, 4096),  
             #функція активації.
            nn.ReLU(inplace=True), 
            #ще один шар Dropout для регуляризації
            nn.Dropout(),  
            nn.Linear(4096, 4096), 
            nn.ReLU(inplace=True),
            #останній шар з виходами num_classes
            nn.Linear(4096, num_classes),  
        )

    def forward(self, x):
        #прохід через згорткові шари
        x = self.features(x) 
        #пулінг
        x = self.avgpool(x)  
        #вирівнювання виходу для лінійних шарів
        x = torch.flatten(x, 1)  
        #прохід через повністю з'єднані шари
        x = self.classifier(x)  
        return x


In [129]:
num_classes = 30  

In [130]:
model = AdjustedCNN(num_classes)

In [131]:
epoch_loss_values = []
step_loss_values = []
#функція втрат
loss_function = nn.CrossEntropyLoss()

In [132]:
#спробуємо тепер використати RMSprop оптимізатор замість Адама
optimizer = torch.optim.RMSprop(adjusted_cnn.parameters(), lr=0.001)

In [133]:
n_epochs = 5 #спочатку тут було 50 епох, але вантажило занадто довго навіть до 10, тому будемо дивитись на 5
n_iters = len(train_data_loader)
n_tab = str(len(str(n_epochs)))

for epoch in range(n_epochs):
    epoch_loss = 0 
    
    for current_batch in train_data_loader:
        
        optimizer.zero_grad()
        
        X_batch, Y_batch = current_batch
        X_batch = X_batch.to(device)
        Y_batch = Y_batch.to(device)
        
        Y_pred = model(X_batch)
        
        loss = loss_function(Y_pred.squeeze(), torch.flatten(Y_batch))
        step_loss_values.append(loss)

        epoch_loss += loss.item() / n_iters
        
        loss.backward()
        optimizer.step()
    
    epoch_loss_values.append(epoch_loss)
            
    print(('epoch {:' + n_tab + '}: epoch_loss: {}').format(epoch+1, epoch_loss)) 

epoch 1: epoch_loss: 3.3716372768833955
epoch 2: epoch_loss: 3.368753797603104
epoch 3: epoch_loss: 3.3779089180928357
epoch 4: epoch_loss: 3.3717273901093674
epoch 5: epoch_loss: 3.364855464899315


In [134]:
del step_loss_values
del epoch_loss_values

In [135]:
#створюємо порожній масив numpy для зберігання прогнозованих міток тренувального набору даних
Y_predicted_train = np.empty([0])

#використовуємо контекст 'torch.no_grad()' для вимкнення розрахунку градієнтів під час прогнозування
with torch.no_grad():
    #ітеруємо по кожному пакету тренувального завантажувача даних
    for current_batch in train_data_loader:
        
        #розділяємо дані та мітки поточного пакету
        X_batch, Y_batch = current_batch
        #конвертуємо дані в FloatTensor та переміщуємо на пристрій
        X_batch = X_batch.type(torch.FloatTensor).to(device)
        #переміщуємо мітки на  пристрій
        Y_batch = Y_batch.to(device)

        #виконуємо пряме поширення моделі для отримання вихідних даних на основі вхідного пакету
        Y_pred = model(X_batch)
        #застосовуємо softmax для перетворення вихідних значень моделі на ймовірності класів
        Y_pred_norm = F.softmax(Y_pred, dim=1)
        #отримуємо індекс класу з найвищою ймовірністю для кожного елементу в пакеті
        Y_pred_label = Y_pred_norm.argmax(dim=1)
        #конвертуємо прогнози в масив numpy та додаємо їх до загального масиву прогнозованих міток
        Y_pred_label = Y_pred_label.detach().cpu().numpy()
        Y_predicted_train = np.append(Y_predicted_train, Y_pred_label)
        
        #видаляємо використані змінні для звільнення пам'яті
        del X_batch
        del Y_batch
        del Y_pred_label
        #очищуємо кеш
        torch.cuda.empty_cache()

#Y_predicted_train.shape


In [136]:
#створюємо порожній масив numpy для зберігання прогнозованих міток тестового набору даних
Y_predicted_test = np.empty([0])

#використовуємо контекст 'torch.no_grad()' для вимкнення розрахунку градієнтів під час прогнозування
with torch.no_grad():
    #ітеруємо по кожному пакету тестового завантажувача даних
    for current_batch in test_data_loader:
        
        #розділяємо дані та мітки поточного пакету
        X_batch, Y_batch = current_batch
        #конвертуємо дані в FloatTensor та переміщуємо на пристрій
        X_batch = X_batch.type(torch.FloatTensor).to(device)
        #переміщуємо мітки на пристрій
        Y_batch = Y_batch.to(device)

        #виконуємо пряме поширення моделі для отримання вихідних даних на основі вхідного пакету
        Y_pred = model(X_batch)
        #застосовуємо softmax для перетворення вихідних значень моделі на ймовірності класів
        Y_pred_norm = F.softmax(Y_pred, dim=1)
        #отримуємо індекс класу з найвищою ймовірністю для кожного елементу в пакеті
        Y_pred_label = Y_pred_norm.argmax(dim=1)
        #конвертуємо прогнози в масив numpy та додаємо їх до загального масиву прогнозованих міток
        Y_pred_label = Y_pred_label.detach().cpu().numpy()
        Y_predicted_test = np.append(Y_predicted_test, Y_pred_label)
        
        #видаляємо використані змінні для звільнення пам'яті
        del X_batch
        del Y_batch
        del Y_pred_label
        #очищуємо кеш
        torch.cuda.empty_cache()
        
#Y_predicted_test.shape


In [137]:
conf_matr_train = multilabel_confusion_matrix(
    y_true=Y_train_labels.squeeze(), 
    y_pred=Y_predicted_train.astype(int)
)

for k, label in enumerate(labels_map.values()):
    print(label)
    print(conf_matr_train[k])

ATR
[[3243   25]
 [  64    2]]
Airbus
[[2719  181]
 [ 408   26]]
Antonov
[[3300    0]
 [  34    0]]
Beechcraft
[[3261    6]
 [  67    0]]
Boeing
[[2482  119]
 [ 703   30]]
Bombardier Aerospace
[[3295    6]
 [  33    0]]
British Aerospace
[[2933  268]
 [ 121   12]]
Canadair
[[2987  213]
 [ 129    5]]
Cessna
[[3201    0]
 [ 133    0]]
Cirrus Aircraft
[[2914  387]
 [  33    0]]
Dassault Aviation
[[3203   64]
 [  66    1]]
Dornier
[[3298    2]
 [  34    0]]
Douglas Aircraft Company
[[3102   99]
 [ 131    2]]
Embraer
[[2738  363]
 [ 215   18]]
Eurofighter
[[3059  242]
 [  31    2]]
Fairchild
[[3200  101]
 [  31    2]]
Fokker
[[2941  293]
 [  87   13]]
Gulfstream Aerospace
[[3146  121]
 [  62    5]]
Ilyushin
[[3297    4]
 [  33    0]]
Lockheed Corporation
[[3266    0]
 [  68    0]]
Lockheed Martin
[[3154  146]
 [  31    3]]
McDonnell Douglas
[[3082   20]
 [ 230    2]]
Panavia
[[3287   13]
 [  34    0]]
Piper
[[3180  121]
 [  31    2]]
Robin
[[3213   88]
 [  29    4]]
Saab
[[3009  258]
 [  63

In [138]:
conf_matr_test = multilabel_confusion_matrix(
    y_true=Y_test_labels.squeeze(), 
    y_pred=Y_predicted_test.astype(int)
)

for k, label in enumerate(labels_map.values()):
    print(label)
    print(conf_matr_test[k])

ATR
[[3248   18]
 [  66    1]]
Airbus
[[2712  188]
 [ 408   25]]
Antonov
[[3295    5]
 [  33    0]]
Beechcraft
[[3254   13]
 [  66    0]]
Boeing
[[2483  116]
 [ 701   33]]
Bombardier Aerospace
[[3294    6]
 [  33    0]]
British Aerospace
[[2923  277]
 [ 117   16]]
Canadair
[[3016  184]
 [ 126    7]]
Cessna
[[3198    1]
 [ 134    0]]
Cirrus Aircraft
[[2901  398]
 [  32    2]]
Dassault Aviation
[[3202   64]
 [  66    1]]
Dornier
[[3295    5]
 [  33    0]]
Douglas Aircraft Company
[[3104   96]
 [ 128    5]]
Embraer
[[2744  356]
 [ 203   30]]
Eurofighter
[[3054  245]
 [  31    3]]
Fairchild
[[3205   94]
 [  34    0]]
Fokker
[[2929  304]
 [  91    9]]
Gulfstream Aerospace
[[3124  142]
 [  63    4]]
Ilyushin
[[3296    3]
 [  34    0]]
Lockheed Corporation
[[3267    0]
 [  66    0]]
Lockheed Martin
[[3170  130]
 [  31    2]]
McDonnell Douglas
[[3082   18]
 [ 231    2]]
Panavia
[[3288   12]
 [  33    0]]
Piper
[[3163  137]
 [  31    2]]
Robin
[[3205   94]
 [  34    0]]
Saab
[[3040  227]
 [  61

In [139]:
conf_matrix = large_confusion_matrix(Y_test_labels, Y_predicted_test, 30, norm=True)

In [140]:
cfm_test = large_confusion_matrix(
    y_true=Y_test_labels, 
    y_pred=Y_predicted_test.astype(int), 
    num_labels=30
)
correct_predictions = np.sum(np.diag(cfm_test))

total_test_samples = len(Y_test_labels) 
accuracy = correct_predictions / total_test_samples

print("Accuracy:", accuracy)

Accuracy: 0.045304530453045305


In [141]:
conf_matrix_train = large_confusion_matrix(Y_train_labels, Y_predicted_train, 30, norm=True)

In [142]:
#використовуємо функцію large_confusion_matrix для створення матриці помилок на основі тренувальних даних
cfm_train = large_confusion_matrix(
    y_true=Y_train_labels,  #реальні мітки з тренувального набору даних
    y_pred=Y_predicted_train.astype(int),  #прогнозовані мітки з тренувального набору даних
    num_labels=30 
)

In [143]:
#обчислюємо кількість правильних прогнозів
#np.diag витягує діагональні елементи матриці (правильні прогнози)
correct_predictions = np.sum(np.diag(cfm_train))

#ділимо кількість правильних прогнозів на загальну кількість тренувальних зразків
total_train_samples = len(Y_train_labels)  #загальна кількість тренувальних зразків
accuracy = correct_predictions / total_train_samples

# Друкуємо розраховану точність.
print("Accuracy:", accuracy)

Accuracy: 0.040491901619676064


In [144]:
#функція для обчислення точності
def calculate_accuracy(y_true, y_pred):
    #розрахунок кількості правильних прогнозів
    correct_predictions = np.sum(y_true == y_pred)
    #визначення загальної кількості зразків
    total_samples = len(y_true)
    #повернення відношення правильних прогнозів до загальної кількості зразків
    return correct_predictions / total_samples

#розрахунок точності для навчального набору даних
#припустимо, що Y_predicted_train та Y_train_labels доступні та мають відповідні форми
train_accuracy = calculate_accuracy(Y_train_labels, Y_predicted_train.astype(int))

#розрахунок точності для тестового набору даних
#припустимо, що Y_predicted_test та Y_test_labels доступні та мають відповідні форми
test_accuracy = calculate_accuracy(Y_test_labels, Y_predicted_test.astype(int))

print("Training Accuracy:", train_accuracy)
print("Test Accuracy:", test_accuracy)

#перевірка на перенавчання (overfitting)
if train_accuracy > test_accuracy:
    if train_accuracy - test_accuracy > 0.1:  
        print("Potential overfitting detected.")
    else:
        print("Slight difference in accuracies but might not be overfitting.")
else:
    print("No significant overfitting detected.")


Training Accuracy: 0.040491901619676064
Test Accuracy: 0.045304530453045305
No significant overfitting detected.


Стало ще гірше, скоріш дуже мало епох було при тренуванні, мережі просто по суті не встигли навчитись.
Але 5 епох ватнажило 40 хвилин для першої CNN, може можна було б бачити краще картину при довшому тренуванні.
Спробуємо поглянути на VGG16

In [146]:
class CustomVGG16(nn.Module):
    def __init__(self, num_classes):
        super(CustomVGG16, self).__init__()
        #завантажимо модель
        self.vgg16 = models.vgg16(pretrained=True)
        #змінимо кількість класів на потрібну під наші дані
        num_features = self.vgg16.classifier[6].in_features
        self.vgg16.classifier[6] = nn.Linear(num_features, num_classes)

    def forward(self, x):
        return self.vgg16(x)

In [148]:
num_classes = 30 
model = CustomVGG16(num_classes)

In [150]:
epoch_loss_values = []
step_loss_values = []
#функція втрат
loss_function = nn.CrossEntropyLoss()
#оптимізатор Адам
optimizer = torch.optim.Adam(model.parameters(), lr=0.005)

In [152]:
n_epochs = 2 #спочатку тут було 50 епох, але вантажило занадто довго навіть до 10, тому будемо дивитись на 5
n_iters = len(train_data_loader)
n_tab = str(len(str(n_epochs)))

for epoch in range(n_epochs):
    epoch_loss = 0 
    
    for current_batch in train_data_loader:
        
        optimizer.zero_grad()
        
        X_batch, Y_batch = current_batch
        X_batch = X_batch.to(device)
        Y_batch = Y_batch.to(device)
        
        Y_pred = model(X_batch)
        
        loss = loss_function(Y_pred.squeeze(), torch.flatten(Y_batch))
        step_loss_values.append(loss)

        epoch_loss += loss.item() / n_iters
        
        loss.backward()
        optimizer.step()
    
    epoch_loss_values.append(epoch_loss)
            
    print(('epoch {:' + n_tab + '}: epoch_loss: {}').format(epoch+1, epoch_loss)) 

KeyboardInterrupt: 

In [ ]:
del step_loss_values
del epoch_loss_values

In [ ]:
#функція для обчислення точності
def calculate_accuracy(y_true, y_pred):
    #розрахунок кількості правильних прогнозів
    correct_predictions = np.sum(y_true == y_pred)
    #визначення загальної кількості зразків
    total_samples = len(y_true)
    #повернення відношення правильних прогнозів до загальної кількості зразків
    return correct_predictions / total_samples

#розрахунок точності для навчального набору даних
#припустимо, що Y_predicted_train та Y_train_labels доступні та мають відповідні форми
train_accuracy = calculate_accuracy(Y_train_labels, Y_predicted_train.astype(int))

#розрахунок точності для тестового набору даних
#припустимо, що Y_predicted_test та Y_test_labels доступні та мають відповідні форми
test_accuracy = calculate_accuracy(Y_test_labels, Y_predicted_test.astype(int))

print("Training Accuracy:", train_accuracy)
print("Test Accuracy:", test_accuracy)

#перевірка на перенавчання (overfitting)
if train_accuracy > test_accuracy:
    if train_accuracy - test_accuracy > 0.1:  
        print("Potential overfitting detected.")
    else:
        print("Slight difference in accuracies but might not be overfitting.")
else:
    print("No significant overfitting detected.")


## Висновок

Проблему з точністю моїх моделей CNN та Adjusted CNN можна спробувати виправити збільшенням кількості епох
та також можна було поєднати тренувальну та валідаційну семпли та тренувати на цьому поєднанні, але ця думка дійшла до мене на середині тренування Adjusted CNN. 

Стосовно VGG16.
VGG16 набагато довше тренувалась (на 5 епохах), змогло видати результат лише перших 2х епох:
#### epoch 1: epoch_loss: 3564.683019480615
#### epoch 2: epoch_loss: 4.1804626122960515

потім kernel почав видавати таймаут.
На 2х епохах вже почав лагати джупайтер та сам компʼютер, прийшлось зупинити (теж вийшло не з першої спроби).

Тенденції втрат: Adjusted CNN і Custom VGG16 показують вищі втрати, ніж Simple CNN, що зазвичай вказує на нижчу продуктивність, але треба зазначити, що величезне значення втрат для першої епохи може бути індикатором проблеми у підготовці даних для моделі або некоректне обрання learning rate, або також у VGG16 велике перенавчання.

Отже, на мою думку VGG16 краще використати у більш складних задачах (де, наприклад, набагато більше класів), бо там це буде зручніше, ніж намагатись
налаштувати свою особисту мережу та робити свою архітектуру.

Але на задачах більш легких краще її не використовувати, бо своя модель тренується швидше, надаючи приблизно такі ж самі результати.

На складних задачах (навіть якщо тренування VGG16 займе багато часу) усе одно буде зекономлено час.
Для більш простих задач це буде нелогічно, легше "покрутити" свою модель, швидше знайдеться помилка та вдосконалення.